# Variable dependiente: Personas Corrupción Intensa (numérica)

## 1. Data

#### 1.1. Import libraries and data

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as  pd, numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
import funciones as fun
import variables_nombres as vn

In [4]:
path = r'..\..\output\data_preprocess\dfs_0_i_pci.csv'
data = pd.read_csv( path )

In [5]:
data

,sexo,facebook,web,propie_muni,mue_pro_1,mue_pro_3,mue_pro_4,mue_pro_5,mue_pro_6,mue_pro_7,...,piagtotfun_f5viv,pimgtotfun_f5viv,tejgtotfun_f5viv,devppimtotfun_f5viv,tipo_control,corrup_intensa,corrup_amplia,per_corrup1,per_corrup2,monto_corrup1
0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,2.0,...,0.000000,0.000000,0.000000,0.000000,2.0,1.0,1.0,1.791759,0.000000,15.851884
1,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,2.0,...,0.000000,10.037975,9.648660,4.230487,2.0,1.0,1.0,1.386294,0.000000,12.060607
2,2.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,2.0,...,0.000000,9.159152,9.159152,4.615121,1.0,1.0,1.0,1.609438,0.000000,15.537098
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,2.079442,0.000000,15.031823
4,2.0,1.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,1.0,...,0.000000,13.399553,0.000000,3.811629,2.0,1.0,1.0,2.197225,1.609438,12.206308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,2.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,13.759167,12.402026,12.402012,4.615106,2.0,0.0,1.0,0.000000,1.098612,0.000000
693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,2.0,0.0,1.0,0.000000,2.197225,0.000000
694,2.0,1.0,1.0,1.0,1.0,0.0,1.0,5.0,0.0,1.0,...,12.105517,12.351281,12.350079,4.613931,2.0,0.0,0.0,0.000000,0.000000,0.000000
695,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.000000,14.749938,14.455608,4.323754,2.0,1.0,1.0,2.833213,0.000000,16.768403


In [6]:
# Borrar otras variables dependientes. Solo nos quedamos con 'monto_corrup2'
data = data.drop( [ 'corrup_intensa', 'corrup_amplia', 'per_corrup2', 
                    'monto_corrup1', 'tipo_control' ], 
                  axis = 1 )

In [7]:
data = data.dropna( axis = 1 )

#### 1.2. Split data into training and test set

In [8]:
np.random.seed( 1234 )
training = np.random.choice( data.index, size=int( len( data )*( 3/4 )), replace = False )

data_train = data.loc[ training, : ]
data_test = data.drop( training, axis = 0 )

## 2. Definir el modelo

In [9]:
import statsmodels.formula.api as smf
import patsy

In [10]:
# varnames =  data.columns.values.tolist()
# varnames = pd.DataFrame( varnames )
# varnames.to_excel( r'..\..\extra\vars_list\pci.xlsx' )

#### 2.1. Construir modelos

In [11]:
%%time

formula_basic = "per_corrup1 ~  dfgdevpiagfun_ct05pgercon +   devppimfun_ct05pgercon +   dfgdevpiagfun_ct05opseg +   tejgfun_ct05trab +   devppimfun_ct05trab +   devppimfun_ct05turi +   piagfun_ct05agro +   pimgfun_ct05agro +   tejgfun_ct05agro +   pimgfun_ct05energia +   tejgfun_ct05energia +   dfgdevpiagfun_ct05energia +   pimgfun_ct05trans +   dfgpimpiafun_ct05trans +   dfgpimpiafun_ct05amb +   dfgdevpiagfun_ct05amb +   devppimfun_ct05san +   devppimfun_ct05viv +   devppimfun_ct05edu +   dfgpimpiafun_ct05protsoc +   devppimfun_ct05protsoc +   piagfun_ct05prevsoc +   tejgfun_ct05prevsoc +   dfgdevpiagfun_ct05prevsoc +   piagfun_ct06pgercon +   tdvgfun_ct06pgercon +   dfgpimpiafun_ct06pgercon +   devppimfun_ct06pgercon +   devppimfun_ct06opseg +   dfgpimpiafun_ct06trab +   devppimfun_ct06trab +   pimgfun_ct06come +   tejgfun_ct06come +   devppimfun_ct06come +   pimgfun_ct06turi +   tdvgfun_ct06turi +   devppimfun_ct06agro +   devppimfun_ct06pesca +   pimgfun_ct06ind +   dfgdevpiagfun_ct06ind +   dfgpimpiafun_ct06trans +   dfgdevpiagfun_ct06trans +   devppimfun_ct06trans +   piagfun_ct06san +   tdvgfun_ct06san +   tejgfun_ct06viv +   tdvgfun_ct06viv +   dfgdevpiagfun_ct06viv +   dfgdevpiagfun_ct06edu +   devppimfun_ct06edu +   devppimft_rdet +   piagrb_reod +   tejgrb_reod +   tdvgrb_reod +   dfgpimpiarb_reod +   devppimrb_reod +   dfgpimpiarb_rooc +   devppimrb_rooc +   devppimrb_dotr +   dfgdevpiagrb_fonc +   devppimrb_fonc +   devppimrb_canr +   tejgct_r00gstcr +   dfgpimpiact_r09gstcr +   tejgct_r19gstcr +   tejgct_r13gstcr +   tdvgct_r13gstcr +   dfgdevpiagct_r07gstcr +   devppimct_r07gstcr +   tejgct_r07gstcp +   piagct_r07srdeu +   pimgct_r07srdeu +   tejgct_r07srdeu +   dfgdevpiagct_r07srdeu +   devppimct_r07srdeu +   dfgpimpiact_r08gstcr +   devppimct_r18gstcr +   tejgge_r00ct05biser +   devppimge_r00ct05biser +   tejgge_r00ct05otgst +   devppimgge_r00ct05otgst +   dfgdevpiagge_r00ct06dotra +   devppimgge_r00ct06otgst +   dfgdevpiagge_r00ct06acanf +   pimgge_r09ct05pobso +   tejgge_r09ct05pobso +   dfgpimpiagge_r09ct05pobso +   dfgdevpiagge_r09ct05pobso +   devppimgge_r09ct05pobso +   pimgge_r09ct05popso +   devppimgge_r09ct05popso +   dfgdevpiagge_r09ct05biser +   pimgge_r09ct05dotra +   tdvgge_r09ct05dotra +   devppimgge_r09ct05dotra +   piagge_r09ct05otgst +   pimgge_r09ct05otgst +   tejgge_r09ct05otgst +   dfgpimpiagge_r09ct05otgst +   devppimgge_r09ct05otgst +   pimgge_r09ct06dotra +   dfgpimpiagge_r09ct06dotra +   devppimgge_r09ct06dotra +   piagge_r09ct06otgst +   tejgge_r09ct06otgst +   tdvgge_r09ct06otgst +   devppimgge_r09ct06otgst +   devppimgge_r09ct06acanf +   dfgdevpiagge_r09ct07sedpu +   dfgpimpiagge_r19ct06dotra +   dfgdevpiagge_r19ct06dotra +   devppimgge_r19ct06dotra +   devppimgge_r13ct05pobso +   tejgge_r13ct05popso +   dfgpimpiagge_r13ct05popso +   devppimgge_r13ct05popso +   devppimgge_r13ct05biser +   piagge_r13ct05dotra +   devppimgge_r13ct05dotra +   tdvgge_r13ct05otgst +   dfgpimpiagge_r13ct05otgst +   devppimgge_r13ct05otgst +   dfgpimpiagge_r13ct06dotra +   dfgdevpiagge_r13ct06dotra +   piagge_r07ct05pobso +   tejgge_r07ct05pobso +   dfgpimpiagge_r07ct05pobso +   dfgdevpiagge_r07ct05pobso +   devppimgge_r07ct05pobso +   dfgpimpiagge_r07ct05popso +   devppimgge_r07ct05popso +   tdvgge_r07ct05biser +   devppimgge_r07ct05biser +   tejgge_r07ct05dotra +   tdvgge_r07ct05dotra +   dfgdevpiagge_r07ct05dotra +   devppimgge_r07ct05dotra +   pimgge_r07ct05otgst +   tejgge_r07ct05otgst +   dfgpimpiagge_r07ct05otgst +   devppimgge_r07ct05otgst +   dfgpimpiage_r08ct05pobso +   devppimge_r08ct05pobso +   dfgpimpiage_r08ct05popso +   devppimge_r08ct05popso +   dfgdevpiagge_r08ct05biser +   piagge_r08ct05dotra +   pimgge_r08ct05dotra +   tejgge_r08ct05dotra +   devppimge_r08ct05dotra +   dfgpimpiagge_r08ct05otgst +   devppimgge_r08ct05otgst +   devppimgge_r08ct06acanf +   dfgdevpiagge_r08ct06acacf +   piagge_r08ct07sedpu +   tejgge_r18ct05pobso +   devppimge_r18ct05pobso +   devppimge_r18ct05popso +   tdvgge_r18ct05dotra +   devppimge_r18ct05dotra +   piagge_r18ct05otgst +   tdvgge_r18ct05otgst +   dfgpimpiagge_r18ct05otgst +   devppimgge_r18ct05otgst +   dfgpimpiagge_r18ct06acacf +   piagge_r18ct07sedpu +   piagkft_reod +   devppimgkft_reod +   tdvgkft_redr +   dfgpimpiagkft_redr +   dfgdevpiagkft_redr +   tejgkft_rooc +   piagkft_dotr +   tejgkft_dotr +   devppimgkft_rdet +   dfgpimpiagkftr07_rdet +   dfgdevpiagkftr07_rdet +   devppimgkftr07_rdet +   dfgpimpiagkftr08_rdet +   dfgdevpiagkftr08_rdet +   dfgdevpiagkftr18_rdet +   tejgfun_f1ct05agro +   tejgfun_f1ct05amb +   tejgfun_f1ct05come +   devppimfun_f1ct05come +   tejgfun_f1ct05cydep +   dfgpimpiafun_f1ct05cydep +   devppimfun_f1ct05cydep +   devppimfun_f1ct05edu +   tejgfun_f1ct05opseg +   devppimfun_f1ct05opseg +   tdvgfun_f1ct05pgercon +   devppimfun_f1ct05pgercon +   dfgdevpiagfun_f1ct05protsoc +   tejgfun_f1ct05salud +   dfgpimpiafun_f1ct05salud +   tejgfun_f1ct05san +   dfgpimpiafun_f1ct05san +   devppimfun_f1ct05san +   piagfun_f1ct05trans +   tejgfun_f1ct05trans +   tdvgfun_f1ct05trans +   devppimfun_f1ct05trans +   devppimfun_f1ct05turi +   tejgfun_f1ct05viv +   dfgpimpiafun_f1ct05viv +   devppimfun_f1ct05viv +   tejgfun_f1ct06agro +   devppimfun_f1ct06agro +   devppimfun_f1ct06amb +   tejgfun_f1ct06come +   dfgdevpiagfun_f1ct06come +   tejgfun_f1ct06cydep +   devppimfun_f1ct06cydep +   piagfun_f1ct06edu +   devppimfun_f1ct06opseg +   dfgpimpiafun_f1ct06pgercon +   devppimfun_f1ct06pgercon +   tejgfun_f1ct06protsoc +   dfgpimpiafun_f1ct06protsoc +   devppimfun_f1ct06protsoc +   tejgfun_f1ct06salud +   devppimfun_f1ct06salud +   tdvgfun_f1ct06san +   dfgpimpiafun_f1ct06san +   devppimfun_f1ct06san +   tejgfun_f1ct06trans +   devppimfun_f1ct06trans +   devppimfun_f1ct06viv +   piagfun_f2ct05agro +   tejgfun_f2ct05agro +   dfgdevpiagfun_f2ct05agro +   devppimfun_f2ct05agro +   dfgpimpiafun_f2ct05amb +   dfgdevpiagfun_f2ct05amb +   devppimfun_f2ct05amb +   dfgpimpiafun_f2ct05come +   devppimfun_f2ct05come +   tejgfun_f2ct05cydep +   dfgpimpiafun_f2ct05cydep +   devppimfun_f2ct05cydep +   dfgpimpiafun_f2ct05edu +   devppimfun_f2ct05edu +   tejgfun_f2ct05energia +   devppimfun_f2ct05energia +   dfgpimpiafun_f2ct05ind +   dfgpimpiafun_f2ct05opseg +   devppimfun_f2ct05opseg +   piagfun_f2ct05pgercon +   tejgfun_f2ct05pgercon +   devppimfun_f2ct05pgercon +   tdvgfun_f2ct05protsoc +   devppimfun_f2ct05protsoc +   piagfun_f2ct05san +   pimgfun_f2ct05san +   tejgfun_f2ct05san +   dfgdevpiagfun_f2ct05san +   devppimfun_f2ct05san +   pimgfun_f2ct05trans +   dfgpimpiafun_f2ct05trans +   devppimfun_f2ct05trans +   tejgfun_f2ct05turi +   devppimfun_f2ct05turi +   devppimfun_f2ct05viv +   piagfun_f2ct06agro +   pimgfun_f2ct06agro +   tejgfun_f2ct06agro +   devppimfun_f2ct06agro +   piagfun_f2ct06amb +   tejgfun_f2ct06amb +   dfgdevpiagfun_f2ct06amb +   devppimfun_f2ct06amb +   piagfun_f2ct06come +   pimgfun_f2ct06come +   tdvgfun_f2ct06come +   dfgpimpiafun_f2ct06come +   dfgdevpiagfun_f2ct06come +   devppimfun_f2ct06come +   tejgfun_f2ct06comunica +   dfgpimpiafun_f2ct06comunica +   dfgdevpiagfun_f2ct06comunica +   pimgfun_f2ct06cydep +   tejgfun_f2ct06cydep +   tdvgfun_f2ct06cydep +   devppimfun_f2ct06cydep +   piagfun_f2ct06edu +   dfgdevpiagfun_f2ct06edu +   devppimfun_f2ct06edu +   piagfun_f2ct06energia +   tejgfun_f2ct06energia +   dfgpimpiafun_f2ct06energia +   piagfun_f2ct06ind +   pimgfun_f2ct06ind +   dfgdevpiagfun_f2ct06ind +   devppimfun_f2ct06ind +   piagfun_f2ct06opseg +   tdvgfun_f2ct06opseg +   dfgdevpiagfun_f2ct06opseg +   devppimfun_f2ct06opseg +   piagfun_f2ct06pgercon +   pimgfun_f2ct06pgercon +   tejgfun_f2ct06pgercon +   dfgdevpiagfun_f2ct06pgercon +   devppimfun_f2ct06pgercon +   piagfun_f2ct06protsoc +   pimgfun_f2ct06protsoc +   tdvgfun_f2ct06protsoc +   dfgdevpiagfun_f2ct06protsoc +   devppimfun_f2ct06protsoc +   piagfun_f2ct06salud +   dfgpimpiafun_f2ct06salud +   dfgdevpiagfun_f2ct06salud +   devppimfun_f2ct06salud +   piagfun_f2ct06san +   tejgfun_f2ct06san +   devppimfun_f2ct06san +   piagfun_f2ct06trans +   tejgfun_f2ct06trans +   dfgpimpiafun_f2ct06trans +   devppimfun_f2ct06trans +   piagfun_f2ct06turi +   tejgfun_f2ct06turi +   tdvgfun_f2ct06turi +   dfgpimpiafun_f2ct06turi +   devppimfun_f2ct06turi +   tejgfun_f2ct06viv +   dfgpimpiafun_f2ct06viv +   devppimfun_f2ct06viv +   devppimfun_f3ct05opseg +   dfgpimpiafun_f3ct05pgercon +   tdvgfun_f3ct06opseg +   dfgpimpiafun_f3ct06opseg +   devppimfun_f3ct06opseg +   tejgfun_f3ct06pgercon +   devppimfun_f3ct06pgercon +   dfgpimpiafun_f4ct05agro +   devppimfun_f4ct05agro +   piagfun_f4ct05amb +   tejgfun_f4ct05amb +   dfgdevpiagfun_f4ct05amb +   devppimfun_f4ct05amb +   dfgpimpiafun_f4ct05come +   dfgdevpiagfun_f4ct05come +   tejgfun_f4ct05cydep +   devppimfun_f4ct05cydep +   tejgfun_f4ct05edu +   devppimfun_f4ct05edu +   tejgfun_f4ct05opseg +   dfgpimpiafun_f4ct05opseg +   devppimfun_f4ct05opseg +   pimgfun_f4ct05pgercon +   tdvgfun_f4ct05pgercon +   devppimfun_f4ct05pgercon +   tejgfun_f4ct05protsoc +   devppimfun_f4ct05protsoc +   tdvgfun_f4ct05salud +   devppimfun_f4ct05salud +   dfgpimpiafun_f4ct05san +   dfgpimpiafun_f4ct05trans +   devppimfun_f4ct05trans +   dfgpimpiafun_f4ct05turi +   dfgdevpiagfun_f4ct05turi +   devppimfun_f4ct05turi +   tdvgfun_f4ct05viv +   dfgpimpiafun_f4ct05viv +   devppimfun_f4ct05viv +   tejgfun_f4ct06agro +   devppimfun_f4ct06agro +   tejgfun_f4ct06amb +   dfgpimpiafun_f4ct06amb +   devppimfun_f4ct06amb +   dfgdevpiagfun_f4ct06come +   devppimfun_f4ct06cydep +   tejgfun_f4ct06edu +   tdvgfun_f4ct06edu +   devppimfun_f4ct06edu +   piagfun_f4ct06pgercon +   dfgpimpiafun_f4ct06pgercon +   dfgdevpiagfun_f4ct06pgercon +   devppimfun_f4ct06pgercon +   dfgpimpiafun_f4ct06protsoc +   devppimfun_f4ct06protsoc +   devppimfun_f4ct06salud +   devppimfun_f4ct06trab +   devppimfun_f4ct06trans +   dfgpimpiafun_f4ct06turi +   devppimfun_f4ct06turi +   devppimfun_f4ct06viv +   tejgfun_f5ct05agro +   devppimfun_f5ct05agro +   dfgdevpiagfun_f5ct05amb +   devppimfun_f5ct05amb +   dfgpimpiafun_f5ct05come +   dfgdevpiagfun_f5ct05comunica +   devppimfun_f5ct05comunica +   piagfun_f5ct05cydep +   tejgfun_f5ct05cydep +   dfgdevpiagfun_f5ct05cydep +   devppimfun_f5ct05cydep +   tdvgfun_f5ct05dpub +   devppimfun_f5ct05dpub +   tejgfun_f5ct05edu +   dfgdevpiagfun_f5ct05edu +   pimgfun_f5ct05energia +   tejgfun_f5ct05energia +   devppimfun_f5ct05energia +   dfgpimpiafun_f5ct05ind +   devppimfun_f5ct05ind +   tejgfun_f5ct05opseg +   tdvgfun_f5ct05opseg +   dfgdevpiagfun_f5ct05opseg +   devppimfun_f5ct05opseg +   devppimfun_f5ct05pgercon +   devppimfun_f5ct05prevsoc +   piagfun_f5ct05protsoc +   tejgfun_f5ct05protsoc +   devppimfun_f5ct05protsoc +   piagfun_f5ct05salud +   tejgfun_f5ct05salud +   devppimfun_f5ct05salud +   piagfun_f5ct05san +   devppimfun_f5ct05trab +   tejgfun_f5ct05trans +   devppimfun_f5ct05trans +   piagfun_f5ct05turi +   dfgdevpiagfun_f5ct05turi +   tejgfun_f5ct05viv +   dfgdevpiagfun_f5ct05viv +   piagfun_f5ct06amb +   tejgfun_f5ct06amb +   devppimfun_f5ct06amb +   piagfun_f5ct06come +   tejgfun_f5ct06cydep +   devppimfun_f5ct06cydep +   devppimfun_f5ct06ind +   piagfun_f5ct06protsoc +   tdvgfun_f5ct06protsoc +   devppimfun_f5ct06protsoc +   devppimfun_f5ct06salud +   tejgfun_f5ct06san +   devppimfun_f5ct06san +   tejgfun_f5ct06trab +   dfgdevpiagfun_f5ct06trab +   piagfun_f5ct06trans +   devppimfun_f5ct06trans +   devppimfun_f5ct06turi +   dfgdevpiagfun_f5ct06viv +   devppimfun_f5ct06viv +   devppimfun_f5energia +   piagfun_f5r07ct05agro +   tejgfun_f5r07ct05agro +   dfgpimpiafun_f5r07ct05agro +   devppimfun_f5r07ct05agro +   dfgdevpiagfun_f5r07ct05amb +   tejgfun_f5r07ct05come +   dfgdevpiagfun_f5r07ct05come +   devppimfun_f5r07ct05come +   piagfun_f5r07ct05comunica +   pimgfun_f5r07ct05comunica +   dfgdevpiagfun_f5r07ct05comunica +   devppimfun_f5r07ct05comunica +   piagfun_f5r07ct05cydep +   tejgfun_f5r07ct05cydep +   dfgdevpiagfun_f5r07ct05cydep +   devppimfun_f5r07ct05cydep +   devppimfun_f5r07ct05dpub +   piagfun_f5r07ct05edu +   tejgfun_f5r07ct05edu +   dfgdevpiagfun_f5r07ct05edu +   devppimfun_f5r07ct05edu +   tejgfun_f5r07ct05energia +   tdvgfun_f5r07ct05energia +   dfgdevpiagfun_f5r07ct05energia +   devppimfun_f5r07ct05energia +   dfgdevpiagfun_f5r07ct05ind +   dfgpimpiafun_f5r07ct05opseg +   tejgfun_f5r07ct05pgercon +   dfgpimpiafun_f5r07ct05pgercon +   pimgfun_f5r07ct05prevsoc +   dfgpimpiafun_f5r07ct05prevsoc +   devppimfun_f5r07ct05prevsoc +   piagfun_f5r07ct05protsoc +   pimgfun_f5r07ct05protsoc +   tejgfun_f5r07ct05protsoc +   dfgpimpiafun_f5r07ct05protsoc +   dfgdevpiagfun_f5r07ct05protsoc +   dfgdevpiagfun_f5r07ct05salud +   devppimfun_f5r07ct05salud +   piagfun_f5r07ct05san +   tejgfun_f5r07ct05san +   dfgdevpiagfun_f5r07ct05san +   devppimfun_f5r07ct05san +   piagfun_f5r07ct05trans +   pimgfun_f5r07ct05trans +   tejgfun_f5r07ct05trans +   dfgdevpiagfun_f5r07ct05trans +   devppimfun_f5r07ct05trans +   piagfun_f5r07ct05turi +   pimgfun_f5r07ct05turi +   tejgfun_f5r07ct05turi +   dfgpimpiafun_f5r07ct05turi +   tejgfun_f5r07ct05viv +   tdvgfun_f5r07ct05viv +   dfgpimpiafun_f5r07ct05viv +   dfgdevpiagfun_f5r07ct05viv +   devppimfun_f5r07ct05viv +   piagfun_f5r07ct06agro +   pimgfun_f5r07ct06agro +   tejgfun_f5r07ct06agro +   dfgpimpiafun_f5r07ct06agro +   devppimfun_f5r07ct06agro +   piagfun_f5r07ct06amb +   tejgfun_f5r07ct06amb +   devppimfun_f5r07ct06amb +   pimgfun_f5r07ct06come +   tejgfun_f5r07ct06come +   tdvgfun_f5r07ct06come +   dfgdevpiagfun_f5r07ct06come +   devppimfun_f5r07ct06come +   piagfun_f5r07ct06comunica +   devppimfun_f5r07ct06comunica +   devppimfun_f5r07ct06cydep +   piagfun_f5r07ct06edu +   pimgfun_f5r07ct06edu +   tejgfun_f5r07ct06edu +   dfgdevpiagfun_f5r07ct06edu +   devppimfun_f5r07ct06edu +   piagfun_f5r07ct06energia +   tejgfun_f5r07ct06energia +   tdvgfun_f5r07ct06energia +   dfgpimpiafun_f5r07ct06energia +   devppimfun_f5r07ct06energia +   pimgfun_f5r07ct06ind +   dfgpimpiafun_f5r07ct06ind +   piagfun_f5r07ct06opseg +   tejgfun_f5r07ct06opseg +   tdvgfun_f5r07ct06opseg +   devppimfun_f5r07ct06opseg +   piagfun_f5r07ct06pgercon +   pimgfun_f5r07ct06pgercon +   tejgfun_f5r07ct06pgercon +   dfgpimpiafun_f5r07ct06pgercon +   devppimfun_f5r07ct06pgercon +   dfgpimpiafun_f5r07ct06prevsoc +   piagfun_f5r07ct06protsoc +   pimgfun_f5r07ct06protsoc +   tejgfun_f5r07ct06protsoc +   devppimfun_f5r07ct06protsoc +   tdvgfun_f5r07ct06salud +   devppimfun_f5r07ct06salud +   devppimfun_f5r07ct06trab +   piagfun_f5r07ct06trans +   dfgpimpiafun_f5r07ct06trans +   tejgfun_f5r07ct06turi +   tdvgfun_f5r07ct06turi +   dfgdevpiagfun_f5r07ct06turi +   devppimfun_f5r07ct06turi +   devppimfun_f5r07ct06viv +   tejgfun_f5r08ct05agro +   tdvgfun_f5r08ct05agro +   dfgdevpiagfun_f5r08ct05agro +   devppimfun_f5r08ct05comunica +   devppimfun_f5r08ct05cydep +   piagfun_f5r08ct05edu +   tejgfun_f5r08ct05edu +   tdvgfun_f5r08ct05edu +   devppimfun_f5r08ct05edu +   piagfun_f5r08ct05energia +   tejgfun_f5r08ct05energia +   dfgdevpiagfun_f5r08ct05energia +   devppimfun_f5r08ct05energia +   piagfun_f5r08ct05opseg +   piagfun_f5r08ct05pgercon +   dfgdevpiagfun_f5r08ct05pgercon +   dfgpimpiafun_f5r08ct05protsoc +   devppimfun_f5r08ct05protsoc +   piagfun_f5r08ct05san +   tejgfun_f5r08ct05san +   tdvgfun_f5r08ct05san +   dfgpimpiafun_f5r08ct05san +   devppimfun_f5r08ct05san +   dfgpimpiafun_f5r08ct05trab +   dfgpimpiafun_f5r08ct05trans +   dfgdevpiagfun_f5r08ct05trans +   devppimfun_f5r08ct05trans +   piagfun_f5r08ct06agro +   pimgfun_f5r08ct06agro +   devppimfun_f5r08ct06agro +   tejgfun_f5r08ct06amb +   tdvgfun_f5r08ct06amb +   dfgpimpiafun_f5r08ct06amb +   devppimfun_f5r08ct06amb +   pimgfun_f5r08ct06come +   tejgfun_f5r08ct06come +   tdvgfun_f5r08ct06come +   dfgpimpiafun_f5r08ct06come +   devppimfun_f5r08ct06come +   piagfun_f5r08ct06comunica +   dfgpimpiafun_f5r08ct06comunica +   devppimfun_f5r08ct06comunica +   piagfun_f5r08ct06cydep +   tejgfun_f5r08ct06cydep +   dfgdevpiagfun_f5r08ct06cydep +   devppimfun_f5r08ct06cydep +   piagfun_f5r08ct06edu +   devppimfun_f5r08ct06edu +   piagfun_f5r08ct06energia +   dfgpimpiafun_f5r08ct06energia +   devppimfun_f5r08ct06energia +   pimgfun_f5r08ct06opseg +   dfgpimpiafun_f5r08ct06opseg +   devppimfun_f5r08ct06opseg +   piagfun_f5r08ct06pgercon +   pimgfun_f5r08ct06pgercon +   dfgdevpiagfun_f5r08ct06pgercon +   devppimfun_f5r08ct06pgercon +   piagfun_f5r08ct06protsoc +   pimgfun_f5r08ct06protsoc +   dfgdevpiagfun_f5r08ct06protsoc +   devppimfun_f5r08ct06protsoc +   tdvgfun_f5r08ct06salud +   dfgdevpiagfun_f5r08ct06salud +   devppimfun_f5r08ct06salud +   dfgdevpiagfun_f5r08ct06san +   devppimfun_f5r08ct06san +   dfgpimpiafun_f5r08ct06trab +   tejgfun_f5r08ct06trans +   devppimfun_f5r08ct06trans +   tejgfun_f5r08ct06turi +   dfgdevpiagfun_f5r08ct06turi +   devppimfun_f5r08ct06turi +   piagfun_f5r08ct06viv +   tejgfun_f5r08ct06viv +   tdvgfun_f5r08ct06viv +   devppimfun_f5r08ct06viv +   piagfun_f5r18ct05agro +   dfgpimpiafun_f5r18ct05agro +   devppimfun_f5r18ct05agro +   piagfun_f5r18ct05amb +   pimgfun_f5r18ct05amb +   tejgfun_f5r18ct05amb +   dfgdevpiagfun_f5r18ct05amb +   devppimfun_f5r18ct05amb +   piagfun_f5r18ct05come +   dfgpimpiafun_f5r18ct05come +   devppimfun_f5r18ct05come +   devppimfun_f5r18ct05comunica +   piagfun_f5r18ct05cydep +   tejgfun_f5r18ct05cydep +   dfgpimpiafun_f5r18ct05cydep +   devppimfun_f5r18ct05cydep +   piagfun_f5r18ct05edu +   tejgfun_f5r18ct05edu +   dfgdevpiagfun_f5r18ct05edu +   devppimfun_f5r18ct05edu +   piagfun_f5r18ct05energia +   dfgpimpiafun_f5r18ct05energia +   devppimfun_f5r18ct05energia +   tejgfun_f5r18ct05ind +   devppimfun_f5r18ct05ind +   piagfun_f5r18ct05opseg +   tejgfun_f5r18ct05opseg +   tdvgfun_f5r18ct05opseg +   dfgdevpiagfun_f5r18ct05opseg +   devppimfun_f5r18ct05opseg +   devppimfun_f5r18ct05pesca +   devppimfun_f5r18ct05pgercon +   dfgdevpiagfun_f5r18ct05prevsoc +   devppimfun_f5r18ct05prevsoc +   piagfun_f5r18ct05protsoc +   pimgfun_f5r18ct05protsoc +   tejgfun_f5r18ct05protsoc +   dfgdevpiagfun_f5r18ct05protsoc +   devppimfun_f5r18ct05protsoc +   tejgfun_f5r18ct05salud +   dfgdevpiagfun_f5r18ct05salud +   devppimfun_f5r18ct05salud +   piagfun_f5r18ct05san +   pimgfun_f5r18ct05san +   devppimfun_f5r18ct05san +   devppimfun_f5r18ct05trab +   piagfun_f5r18ct05trans +   tejgfun_f5r18ct05trans +   devppimfun_f5r18ct05trans +   piagfun_f5r18ct05turi +   tejgfun_f5r18ct05turi +   tdvgfun_f5r18ct05turi +   dfgdevpiagfun_f5r18ct05turi +   devppimfun_f5r18ct05turi +   piagfun_f5r18ct05viv +   tejgfun_f5r18ct05viv +   dfgpimpiafun_f5r18ct05viv +   dfgdevpiagfun_f5r18ct05viv +   devppimfun_f5r18ct05viv +   tejgfun_f5r18ct06agro +   dfgpimpiafun_f5r18ct06agro +   devppimfun_f5r18ct06agro +   devppimfun_f5r18ct06amb +   devppimfun_f5r18ct06come +   devppimfun_f5r18ct06comunica +   devppimfun_f5r18ct06cydep +   piagfun_f5r18ct06edu +   devppimfun_f5r18ct06edu +   dfgpimpiafun_f5r18ct06ind +   piagfun_f5r18ct06opseg +   pimgfun_f5r18ct06opseg +   tejgfun_f5r18ct06opseg +   dfgdevpiagfun_f5r18ct06opseg +   devppimfun_f5r18ct06opseg +   tejgfun_f5r18ct06pesca +   dfgpimpiafun_f5r18ct06pesca +   devppimfun_f5r18ct06pgercon +   piagfun_f5r18ct06prevsoc +   devppimfun_f5r18ct06prevsoc +   pimgfun_f5r18ct06protsoc +   tejgfun_f5r18ct06protsoc +   devppimfun_f5r18ct06protsoc +   piagfun_f5r18ct06salud +   dfgpimpiafun_f5r18ct06salud +   devppimfun_f5r18ct06salud +   devppimfun_f5r18ct06trab +   dfgpimpiafun_f5r18ct06trans +   dfgdevpiagfun_f5r18ct06trans +   devppimfun_f5r18ct06trans +   piagfun_f5r18ct06viv +   tejgfun_f5r18ct06viv +   dfgdevpiagfun_f5r18ct06viv +   devppimfun_f5r18ct06viv +   piagtotfun_f1agro +   pimgtotfun_f1agro +   devppimtotfun_f1agro +   tejgtotfun_f1amb +   tdvgtotfun_f1amb +   devppimtotfun_f1amb +   tdvgtotfun_f1come +   dfgdevpiagtotfun_f1cydep +   piagtotfun_f1edu +   tejgtotfun_f1edu +   dfgpimpiatotfun_f1edu +   devppimtotfun_f1edu +   tejgtotfun_f1energia +   devppimtotfun_f1energia +   piagtotfun_f1opseg +   pimgtotfun_f1opseg +   tdvgtotfun_f1opseg +   dfgpimpiatotfun_f1pgercon +   piagtotfun_f1prevsoc +   devppimtotfun_f1prevsoc +   devppimtotfun_f1protsoc +   piagtotfun_f1salud +   dfgpimpiatotfun_f1salud +   devppimtotfun_f1salud +   piagtotfun_f1san +   dfgdevpiagtotfun_f1trab +   pimgtotfun_f1trans +   dfgdevpiagtotfun_f1turi +   devppimtotfun_f1turi +   dfgdevpiagtotfun_f1viv +   piagtotfun_f2agro +   dfgdevpiagtotfun_f2agro +   devppimtotfun_f2agro +   piagtotfun_f2come +   dfgdevpiagtotfun_f2come +   piagtotfun_f2comunica +   dfgpimpiatotfun_f2comunica +   piagtotfun_f2cydep +   pimgtotfun_f2cydep +   dfgpimpiatotfun_f2cydep +   dfgpimpiatotfun_f2dpub +   devppimtotfun_f2dpub +   tejgtotfun_f2edu +   devppimtotfun_f2edu +   piagtotfun_f2energia +   devppimtotfun_f2energia +   pimgtotfun_f2ind +   dfgpimpiatotfun_f2pgercon +   dfgdevpiagtotfun_f2prevsoc +   devppimtotfun_f2prevsoc +   piagtotfun_f2protsoc +   tejgtotfun_f2protsoc +   dfgdevpiagtotfun_f2protsoc +   tdvgtotfun_f2salud +   dfgdevpiagtotfun_f2salud +   devppimtotfun_f2salud +   piagtotfun_f2san +   pimgtotfun_f2san +   tejgtotfun_f2san +   tdvgtotfun_f2san +   dfgdevpiagtotfun_f2san +   devppimtotfun_f2san +   devppimtotfun_f2trab +   tejgtotfun_f2trans +   dfgpimpiatotfun_f2trans +   devppimtotfun_f2trans +   pimgtotfun_f2turi +   dfgpimpiatotfun_f2turi +   dfgdevpiagtotfun_f2turi +   pimgtotfun_f2viv +   tejgtotfun_f2viv +   dfgpimpiatotfun_f2viv +   piagtotfun_f3agro +   tejgtotfun_f3agro +   tdvgtotfun_f3agro +   devppimtotfun_f3agro +   tejgtotfun_f3amb +   devppimtotfun_f3amb +   devppimtotfun_f3come +   dfgpimpiatotfun_f3comunica +   tejgtotfun_f3cydep +   dfgpimpiatotfun_f3cydep +   devppimtotfun_f3cydep +   piagtotfun_f3edu +   pimgtotfun_f3edu +   tejgtotfun_f3edu +   dfgdevpiagtotfun_f3edu +   devppimtotfun_f3edu +   tejgtotfun_f3energia +   dfgpimpiatotfun_f3energia +   devppimtotfun_f3energia +   dfgpimpiatotfun_f3ind +   tejgtotfun_f3opseg +   tdvgtotfun_f3opseg +   dfgpimpiatotfun_f3pesca +   piagtotfun_f3pgercon +   devppimtotfun_f3pgercon +   tejgtotfun_f3protsoc +   dfgpimpiatotfun_f3protsoc +   devppimtotfun_f3protsoc +   dfgpimpiatotfun_f3salud +   devppimtotfun_f3salud +   piagtotfun_f3san +   tejgtotfun_f3san +   tdvgtotfun_f3san +   devppimtotfun_f3san +   dfgpimpiatotfun_f3trab +   devppimtotfun_f3trab +   pimgtotfun_f3trans +   devppimtotfun_f3trans +   devppimtotfun_f3turi +   piagtotfun_f3viv +   tejgtotfun_f3viv +   tdvgtotfun_f3viv +   devppimtotfun_f3viv +   dfgpimpiatotfun_f4agro +   piagtotfun_f4amb +   pimgtotfun_f4amb +   devppimtotfun_f4come +   tejgtotfun_f4cydep +   dfgpimpiatotfun_f4cydep +   devppimtotfun_f4cydep +   tejgtotfun_f4energia +   tdvgtotfun_f4energia +   dfgpimpiatotfun_f4energia +   devppimtotfun_f4energia +   tejgtotfun_f4opseg +   devppimtotfun_f4opseg +   piagtotfun_f4pgercon +   tdvgtotfun_f4pgercon +   devppimtotfun_f4prevsoc +   tejgtotfun_f4protsoc +   dfgpimpiatotfun_f4protsoc +   piagtotfun_f4salud +   pimgtotfun_f4salud +   devppimtotfun_f4salud +   tdvgtotfun_f4san +   devppimtotfun_f4san +   dfgpimpiatotfun_f4trab +   devppimtotfun_f4trab +   dfgdevpiagtotfun_f4turi +   devppimtotfun_f4turi +   dfgpimpiatotfun_f4viv +   dfgdevpiagtotfun_f5agro +   devppimtotfun_f5agro +   piagtotfun_f5amb +   tejgtotfun_f5amb +   devppimtotfun_f5amb +   pimgtotfun_f5come +   tejgtotfun_f5come +   dfgpimpiatotfun_f5come +   devppimtotfun_f5come +   pimgtotfun_f5comunica +   dfgdevpiagtotfun_f5comunica +   piagtotfun_f5cydep +   dfgpimpiatotfun_f5cydep +   devppimtotfun_f5cydep +   piagtotfun_f5dpub +   pimgtotfun_f5dpub +   tejgtotfun_f5dpub +   dfgpimpiatotfun_f5dpub +   devppimtotfun_f5dpub +   devppimtotfun_f5edu +   tejgtotfun_f5ind +   devppimtotfun_f5ind +   piagtotfun_f5opseg +   pimgtotfun_f5opseg +   dfgdevpiagtotfun_f5opseg +   devppimtotfun_f5opseg +   dfgdevpiagtotfun_f5pgercon +   dfgpimpiatotfun_f5prevsoc +   tdvgtotfun_f5protsoc +   devppimtotfun_f5protsoc +   devppimtotfun_f5r07agro +   piagtotfun_f5r07amb +   dfgpimpiatotfun_f5r07amb +   devppimtotfun_f5r07amb +   piagtotfun_f5r07come +   dfgdevpiagtotfun_f5r07come +   tejgtotfun_f5r07comunica +   dfgpimpiatotfun_f5r07comunica +   piagtotfun_f5r07cydep +   tejgtotfun_f5r07cydep +   dfgpimpiatotfun_f5r07cydep +   devppimtotfun_f5r07cydep +   pimgtotfun_f5r07energia +   dfgdevpiagtotfun_f5r07energia +   piagtotfun_f5r07ind +   dfgpimpiatotfun_f5r07ind +   dfgdevpiagtotfun_f5r07ind +   tejgtotfun_f5r07opseg +   tdvgtotfun_f5r07opseg +   dfgdevpiagtotfun_f5r07opseg +   devppimtotfun_f5r07opseg +   piagtotfun_f5r07pesca +   tdvgtotfun_f5r07pesca +   dfgdevpiagtotfun_f5r07pesca +   devppimtotfun_f5r07pesca +   piagtotfun_f5r07pgercon +   dfgdevpiagtotfun_f5r07pgercon +   devppimtotfun_f5r07pgercon +   tejgtotfun_f5r07prevsoc +   tdvgtotfun_f5r07protsoc +   dfgdevpiagtotfun_f5r07protsoc +   devppimtotfun_f5r07protsoc +   piagtotfun_f5r07salud +   dfgdevpiagtotfun_f5r07salud +   piagtotfun_f5r07san +   pimgtotfun_f5r07san +   tejgtotfun_f5r07san +   dfgpimpiatotfun_f5r07san +   devppimtotfun_f5r07san +   pimgtotfun_f5r07trab +   dfgdevpiagtotfun_f5r07trab +   devppimtotfun_f5r07trab +   devppimtotfun_f5r07trans +   pimgtotfun_f5r07turi +   devppimtotfun_f5r07turi +   tejgtotfun_f5r07viv +   tdvgtotfun_f5r07viv +   dfgpimpiatotfun_f5r07viv +   devppimtotfun_f5r07viv +   piagtotfun_f5r08agro +   dfgdevpiagtotfun_f5r08agro +   devppimtotfun_f5r08agro +   piagtotfun_f5r08amb +   tejgtotfun_f5r08amb +   dfgdevpiagtotfun_f5r08amb +   devppimtotfun_f5r08amb +   piagtotfun_f5r08come +   tejgtotfun_f5r08come +   dfgdevpiagtotfun_f5r08cydep +   tdvgtotfun_f5r08dpub +   dfgdevpiagtotfun_f5r08dpub +   devppimtotfun_f5r08dpub +   pimgtotfun_f5r08edu +   dfgdevpiagtotfun_f5r08edu +   devppimtotfun_f5r08edu +   piagtotfun_f5r08energia +   devppimtotfun_f5r08energia +   piagtotfun_f5r08ind +   dfgdevpiagtotfun_f5r08ind +   devppimtotfun_f5r08ind +   tdvgtotfun_f5r08opseg +   dfgpimpiatotfun_f5r08opseg +   devppimtotfun_f5r08opseg +   dfgpimpiatotfun_f5r08pgercon +   devppimtotfun_f5r08pgercon +   dfgpimpiatotfun_f5r08prevsoc +   devppimtotfun_f5r08prevsoc +   dfgdevpiagtotfun_f5r08protsoc +   piagtotfun_f5r08salud +   tejgtotfun_f5r08salud +   tdvgtotfun_f5r08salud +   devppimtotfun_f5r08salud +   piagtotfun_f5r08san +   tejgtotfun_f5r08san +   tdvgtotfun_f5r08san +   dfgpimpiatotfun_f5r08san +   devppimtotfun_f5r08trab +   dfgpimpiatotfun_f5r08trans +   tejgtotfun_f5r08turi +   devppimtotfun_f5r08turi +   dfgpimpiatotfun_f5r08viv +   devppimtotfun_f5r08viv +   devppimtotfun_f5r18agro +   pimgtotfun_f5r18amb +   dfgpimpiatotfun_f5r18amb +   devppimtotfun_f5r18amb +   dfgpimpiatotfun_f5r18come +   dfgdevpiagtotfun_f5r18come +   devppimtotfun_f5r18come +   piagtotfun_f5r18cydep +   tdvgtotfun_f5r18cydep +   tejgtotfun_f5r18dpub +   dfgdevpiagtotfun_f5r18dpub +   devppimtotfun_f5r18dpub +   tdvgtotfun_f5r18edu +   dfgpimpiatotfun_f5r18edu +   devppimtotfun_f5r18ind +   tdvgtotfun_f5r18opseg +   devppimtotfun_f5r18opseg +   piagtotfun_f5r18pesca +   pimgtotfun_f5r18pesca +   piagtotfun_f5r18pgercon +   devppimtotfun_f5r18pgercon +   piagtotfun_f5r18prevsoc +   pimgtotfun_f5r18prevsoc +   devppimtotfun_f5r18protsoc +   devppimtotfun_f5r18salud +   dfgdevpiagtotfun_f5r18san +   devppimtotfun_f5r18san +   tejgtotfun_f5r18trab +   dfgdevpiagtotfun_f5r18trab +   devppimtotfun_f5r18trab +   devppimtotfun_f5r18turi +   pimgtotfun_f5r18viv +   dfgpimpiatotfun_f5r18viv +   devppimtotfun_f5r18viv +   tejgtotfun_f5salud +   dfgdevpiagtotfun_f5salud +   devppimtotfun_f5salud +   piagtotfun_f5trab +   devppimtotfun_f5trab +   tejgtotfun_f5trans +   tejgtotfun_f5turi +   tdvgtotfun_f5turi +   dfgpimpiatotfun_f5turi +   dfgdevpiagtotfun_f5turi +   devppimtotfun_f5turi +   piagtotfun_f5viv +   pimgtotfun_f5viv +   tejgtotfun_f5viv +   devppimtotfun_f5viv"
y_basic_train, x_basic_train = patsy.dmatrices( formula_basic, data_train, return_type = 'dataframe' )
y_basic_test, x_basic_test = patsy.dmatrices( formula_basic, data_test, return_type = 'dataframe' )
p_basic = x_basic_train.shape[ 1 ]

Wall time: 2.34 s


#### 2.2. Generar variables dependientes

In [12]:
Y_train = data_train[ 'per_corrup1' ]
Y_test = data_test[ 'per_corrup1' ]

In [13]:
p_basic

1043

## 3. OLS

In [14]:
x_basic_train.head()

,Intercept,dfgdevpiagfun_ct05pgercon,devppimfun_ct05pgercon,dfgdevpiagfun_ct05opseg,tejgfun_ct05trab,devppimfun_ct05trab,devppimfun_ct05turi,piagfun_ct05agro,pimgfun_ct05agro,tejgfun_ct05agro,...,tejgtotfun_f5trans,tejgtotfun_f5turi,tdvgtotfun_f5turi,dfgpimpiatotfun_f5turi,dfgdevpiagtotfun_f5turi,devppimtotfun_f5turi,piagtotfun_f5viv,pimgtotfun_f5viv,tejgtotfun_f5viv,devppimtotfun_f5viv
339,1.0,-0.527254,4.289474,0.307406,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,14.711309,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,12.190964,12.027408,4.453322
549,1.0,0.089348,4.511438,0.000000,0.0,0.0,4.615121,0.000000,0.000000,0.0,...,10.787709,10.308986,10.308986,0.030,0.030000,4.615121,0.000000,0.000000,0.000000,0.000000
255,1.0,0.129991,4.526656,0.000000,0.0,0.0,0.000000,0.000000,11.512935,0.0,...,0.000000,0.000000,10.565009,0.045,0.038754,4.467280,12.611541,12.429220,0.000000,3.921102
83,1.0,0.388732,4.291729,-0.171162,0.0,0.0,0.000000,9.179984,8.218518,0.0,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,10.007893,0.000000,0.000000
673,1.0,-9.438865,4.425671,-0.185236,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,13.906523,0.000000,4.553912


#### 3.1. Ajustar el modelo

In [15]:
fit_lm_basic = smf.ols( formula_basic, data = data_train ).fit()

#### 3.2. Calcular out-of-sample MSE y Standard Error

In [16]:
# Only MSE

yhat_lm_basic = fit_lm_basic.predict( data_test )
print("The mean squared error (MSE) using the basic model is equal to:", ( ( Y_test - yhat_lm_basic )**2 ).mean() )

The mean squared error (MSE) using the basic model is equal to: 3.7819194678644594


In [17]:
# MSE and SE

resid_basic  = ( Y_test - yhat_lm_basic )**2

MSE_lm_basic = sm.OLS( resid_basic , np.ones( resid_basic.shape[ 0 ] ) ).fit().summary2().tables[ 1 ].iloc[ 0, 0:2 ]
MSE_lm_basic

Coef.       3.781919
Std.Err.    0.418043
Name: const, dtype: float64

#### 3.3. Calcular R Cuadrado out-of-sample

In [18]:
R2_lm_basic = 1 - ( MSE_lm_basic[ 0 ]/Y_test.var() )
print( f"The R^2 using the basic model is equal to: { R2_lm_basic }" )

The R^2 using the basic model is equal to: -3.426307426698995


## 4. Lasso, Ridge and Elastic Net

#### 4.1. Theoretical Lasso from hdm package

In [19]:
import hdmpy as hdm

In [20]:
fit_rlasso = hdm.rlasso( x_basic_train.to_numpy() , Y_train.to_numpy().reshape( Y_train.size , 1 ) , post = False )
fit_rlasso_post = hdm.rlasso( x_basic_train.to_numpy() , Y_train.to_numpy().reshape( Y_train.size , 1 ) , post = True )

In [21]:
# Getting mean of each variable
meanx = x_basic_test.mean( axis = 0 ).values.\
                        reshape( x_basic_test.shape[ 1 ] , 1 )

# Reducing the mean
new_x1 = x_basic_test.to_numpy() - \
                    (np.ones( ( x_basic_test.shape[ 0 ] , 1 ) ) @ meanx.T)

# Getting the significant variables
x1_est_rlasso = new_x1[ :, fit_rlasso.est['index'].iloc[:, 0].to_list()]

# Getting the coef. from significant variables
beta_rlasso = fit_rlasso.est['beta'].loc[ fit_rlasso.est['index'].\
                                     iloc[:, 0].to_list(), ].to_numpy()

# yhat
yhat_rlasso = (x1_est_rlasso @ beta_rlasso) + np.mean( Y_test.to_numpy() )
residuals_rlasso = Y_test.to_numpy().reshape( Y_test.to_numpy().size, 1)  - yhat_rlasso

In [22]:
# Getting mean of each variable
meanx = x_basic_test.mean( axis = 0 ).values.\
                        reshape(x_basic_test.shape[ 1 ] , 1 )

# Reducing the mean
new_x1 = x_basic_test.to_numpy() - \
                    (np.ones( (x_basic_test.shape[ 0 ] , 1 ) ) @ meanx.T)

# Getting the significant variables
x1_est_rlasso_post = new_x1[ :, fit_rlasso_post.est['index'].iloc[:, 0].to_list()]

# Getting the coef. from significant variables
beta_rlasso_post = fit_rlasso_post.est['beta'].loc[ fit_rlasso_post.est['index'].\
                                     iloc[:, 0].to_list(), ].to_numpy()

# yhat
yhat_rlasso_post = (x1_est_rlasso_post @ beta_rlasso_post) + np.mean( Y_test.to_numpy() )
residuals_rlasso_post = Y_test.to_numpy().reshape( Y_test.to_numpy().size, 1)  - yhat_rlasso_post

In [23]:
MSE_lasso = sm.OLS( ( residuals_rlasso )**2 , np.ones( yhat_rlasso.size )  ).fit().summary2().tables[1].round(3)
MSE_lasso_post = sm.OLS( ( residuals_rlasso_post )**2  , np.ones( yhat_rlasso_post.size )  ).fit().summary2().tables[1].round(3)

R2_lasso = 1 - MSE_lasso.iloc[0, 0]/ np.var( Y_test )
R2_lasso_post = 1 - MSE_lasso_post.iloc[0, 0]/ np.var( Y_test )

print( f"The R^2 using the basic model is equal to {R2_lasso},for lasso and {R2_lasso_post} for post-lasso") # R^2 lasso/post-lasso (basic model) 

The R^2 using the basic model is equal to 0.057132613114234676,for lasso and 0.024173453522722377 for post-lasso


#### 4.2. Cross Validated Lasso, Ridge and Elastic Net

In [24]:
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV

In [25]:
fit_lasso_cv = LassoCV( cv = 10, fit_intercept = True, normalize = False, random_state = 0 ).fit( x_basic_train, Y_train )
fit_ridge = RidgeCV( cv = 10, fit_intercept = True, normalize = False, scoring = None ).fit( x_basic_train, Y_train )
fit_elnet = ElasticNetCV( cv = 10, fit_intercept = True, normalize = False, random_state = 0 ).fit( x_basic_train, Y_train )

yhat_lasso_cv = fit_lasso_cv.predict( x_basic_test )
yhat_ridge = fit_ridge.predict( x_basic_test )
yhat_elnet = fit_elnet.predict( x_basic_test )

residual_lasso = ( yhat_lasso_cv - Y_test )**2
residual_ridge = ( yhat_ridge - Y_test )**2
residual_elnet = ( yhat_elnet - Y_test )**2

MSE_lasso_cv = sm.OLS( residual_lasso, np.ones( Y_test.size )).fit().summary2().tables[ 1 ].round( 3 )
MSE_ridge = sm.OLS( residual_ridge, np.ones( Y_test.size )).fit().summary2().tables[ 1 ].round( 3 )
MSE_elnet = sm.OLS( residual_elnet, np.ones( Y_test.size )).fit().summary2().tables[ 1 ].round( 3 )

R2_lasso_cv = 1 - MSE_lasso_cv.iloc[ 0, 0 ] / np.var( Y_test )
R2_ridge = 1 - MSE_ridge.iloc[ 0, 0 ]  / np.var( Y_test )
R2_elnet = 1 - MSE_elnet.iloc[ 0, 0 ]  / np.var( Y_test )

print( "R^2 using cross-validation for lasso, ridge, and elastic net in the basic model: {:.5f}, {:.5f}, {:.5f}".format( R2_lasso_cv, R2_ridge, R2_elnet ) )

R^2 using cross-validation for lasso, ridge, and elastic net in the basic model: 0.03477, -2.26767, 0.03477


## 5. Non Linear Models

#### 5.1. Regression Trees

In [26]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

In [27]:
# Fit the model

fit_trees = DecisionTreeRegressor( random_state = 0, min_impurity_decrease = 0.001 )
fit_trees.fit( x_basic_train, y_basic_train )

DecisionTreeRegressor(min_impurity_decrease=0.001, random_state=0)

In [28]:
# Plot the tree

# from sklearn.tree import plot_tree

# plt.figure( figsize=( 30, 20 ) )
# plot_tree( fit_trees, filled = True )
# plt.show()

In [29]:
# Determine the optimar complexity of the regression tree

s = pd.DataFrame( fit_trees.cost_complexity_pruning_path( y_basic_train, x_basic_train ) )
s.head()

,ccp_alphas,impurities
0,0.000000,7.648052
1,0.008004,7.656056
2,0.012372,7.668428
3,0.012893,7.694214
4,0.013613,7.707827


In [30]:
### PREGUNTA: DE DÓNDE SALE ESE VALOR DE ALPHA

In [31]:
# Prune the tree

fit_prunnedtree = DecisionTreeRegressor( ccp_alpha = 0.00188444410871555 )
fit_prunnedtree.fit( x_basic_train, y_basic_train )

DecisionTreeRegressor(ccp_alpha=0.00188444410871555)

In [32]:
# Plot the prunned tree

# plot_tree(fit_prunnedtree, filled=True)
# plt.show()

In [33]:
# Calculate MSE and R2 for prunned tree

y_hat_pt = fit_prunnedtree.predict( x_basic_test )
residual_pt = ( y_hat_pt - Y_test )**2
MSE_pt = sm.OLS( residual_pt, np.ones( y_hat_pt.size )).fit().summary2().tables[ 1 ].round( 3 )
R2_pt = 1 - MSE_pt.iloc[ 0, 0 ]/np.var( Y_test )
print( f"R^2 of the pruned tree: { R2_pt }" )

R^2 of the pruned tree: -0.7597836996182512


#### 5.2. Random Forest and Booested Trees

In [34]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor 

In [35]:
# random forest
fit_rf = RandomForestRegressor( n_estimators = 2000, min_samples_leaf = 5).fit( x_basic_train, Y_train )

# boosting
fit_boost = GradientBoostingRegressor( loss = 'ls', learning_rate = 0.01, n_estimators = 1000, max_depth = 2, subsample = 0.5).fit( x_basic_train, Y_train )

# Evaluating the methods
yhat_rf = fit_rf.predict( x_basic_test )
residual_rf = ( yhat_rf - Y_test )**2
yhat_boost = fit_boost.predict( x_basic_test )
residual_bst = ( yhat_boost - Y_test )**2

# Calculate MSE
MSE_rf = sm.OLS( residual_pt, np.ones( yhat_rf.size )).fit().summary2().tables[ 1 ].round( 3 )
MSE_bst = sm.OLS( residual_bst, np.ones( yhat_boost.size )).fit().summary2().tables[ 1 ].round( 3 )

# Calculate R2
R2_rf = 1 - MSE_rf.iloc[ 0, 0 ] / Y_test.var()
R2_boost = 1 - MSE_bst.iloc[ 0, 0 ] / Y_test.var()

In [36]:
print("R^2 of the random forest and boosted trees:{:.5f}, {:.5f}".format( R2_rf, R2_boost ))

R^2 of the random forest and boosted trees:-0.74973, 0.08008


## 6. Resultados

In [37]:
table = pd.DataFrame(columns=["MSE", "S.E for MSE", "R-squared"]) 
table.loc[0]  = [MSE_lm_basic[0], MSE_lm_basic[1], R2_lm_basic]
table.loc[1]  = [MSE_lasso.iloc[0, 0], MSE_lasso.iloc[0, 1], R2_lasso]
table.loc[2]  = [MSE_lasso.iloc[0, 0], MSE_lasso_post.iloc[0, 1], R2_lasso_post]
table.loc[3]  = [MSE_lasso_cv.iloc[0, 0], MSE_lasso_cv.iloc[0, 1], R2_lasso_cv]
table.loc[4]  = [MSE_ridge.iloc[0, 0], MSE_ridge.iloc[0, 1], R2_ridge]
table.loc[5]  = [MSE_elnet.iloc[0, 0], MSE_elnet.iloc[0, 1], R2_elnet]
table.loc[6] = [MSE_rf.iloc[0, 0], MSE_rf.iloc[0, 1], R2_rf]
table.loc[7] = [MSE_bst.iloc[0, 0], MSE_bst.iloc[0, 1], R2_boost]
table.loc[8] = [MSE_pt.iloc[0, 0], MSE_pt.iloc[0, 1], R2_pt]
models_row = [ "Least Squares (basic)", "Lasso", "Post_Lasso", "Cross-Validated lasso", 
               "Cross-Validated ridge", "Cross-Validated elnet", "Random Forest", 
               "Boosted Trees", "Pruned Tree" ]
table.insert( 0, "Models", models_row )
table

,Models,MSE,S.E for MSE,R-squared
0,Least Squares (basic),3.781919,0.418043,-3.426307
1,Lasso,0.801000,0.055000,0.057133
2,Post_Lasso,0.801000,0.057000,0.024173
3,Cross-Validated lasso,0.820000,0.058000,0.034767
4,Cross-Validated ridge,2.776000,0.316000,-2.267665
5,Cross-Validated elnet,0.820000,0.058000,0.034767
6,Random Forest,1.495000,0.135000,-0.749728
7,Boosted Trees,0.786000,0.064000,0.080076
8,Pruned Tree,1.495000,0.135000,-0.759784
